In [ ]:
pip install mediapipe

In [6]:
import cv2
import mediapipe as mp
import math
import numpy as np

## Basico

In [5]:
# Inicialización
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

# Captura de video
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Conversión a RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Procesar el frame
    results = hands.process(frame_rgb)

    # Dibujo de puntos clave
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Mostrar el video con las manos detectadas
    cv2.imshow('Hand Detection', frame)
    if cv2.waitKey(1) & 0xFF == 27:  # Salir con ESC
        break

cap.release()
cv2.destroyAllWindows()


## Dibujar

In [4]:
import cv2
import mediapipe as mp
import math

# Inicializar MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

# Función para calcular la distancia euclidiana
def calculate_distance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

# Pintar un área alrededor de un punto en rojo
def paint_pixels(image, center, size=5):
    x, y = center
    for i in range(-size, size + 1):
        for j in range(-size, size + 1):
            if 0 <= y + i < image.shape[0] and 0 <= x + j < image.shape[1]:
                image[y + i, x + j] = (0, 0, 255)

# Dibujar el botón "Limpiar"
def draw_button(image):
    h, w, _ = image.shape
    top_left = (w - 200, 10)
    bottom_right = (w - 20, 60)
    cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), -1)
    cv2.putText(image, "Limpiar", (w - 180, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    return top_left, bottom_right

# Capturar video
cap = cv2.VideoCapture(0)
dibujo = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks = hand_landmarks.landmark
            h, w, _ = frame.shape
            thumb_tip = (landmarks[4].x, landmarks[4].y)
            index_tip = (landmarks[8].x, landmarks[8].y)
            thumb_pixel = (int(thumb_tip[0] * w), int(thumb_tip[1] * h))
            index_pixel = (int(index_tip[0] * w), int(index_tip[1] * h))
            distance = calculate_distance(thumb_tip, index_tip)
            if distance < 0.10:
                dibujo.append(index_pixel)
            for i in dibujo:
                paint_pixels(frame, i)
            if top_left[0] <= index_pixel[0] <= bottom_right[0] and top_left[1] <= index_pixel[1] <= bottom_right[1]:
                dibujo.clear()  # Limpiar los puntos dibujados
    # Dibujar el botón "Limpiar"
    top_left, bottom_right = draw_button(frame)

    # Detectar clic en el botón
    if cv2.waitKey(1) & 0xFF == ord('c'):  # Simula un clic al presionar 'c'
        dibujo.clear()

    cv2.imshow('Hand Detection', frame)
    if cv2.waitKey(1) & 0xFF == 27:  # Salir con ESC
        break

cap.release()
cv2.destroyAllWindows()


## Vriante 1

In [28]:
import cv2
import mediapipe as mp
import math
import numpy as np

# Inicializar MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

# Función para calcular la distancia euclidiana
def calculate_distance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def draw_keyboard(frame, key_size=50):
    h, w, _ = frame.shape
    keys = "ABCDEFGHIJKLMNOPQRSTUVWXYZ<3 "
    keyboard = []
    start_x, start_y = 30, h - 200  # Coordenadas iniciales del teclado
    for i, key in enumerate(keys):
        x = start_x + (i % 10) * (key_size + 5)  # 10 teclas por fila
        y = start_y + (i // 10) * (key_size + 5)
        top_left = (x, y)
        bottom_right = (x + key_size, y + key_size)
        cv2.rectangle(frame, top_left, bottom_right, (255, 255, 255), 2)
        if key == "❤":
            # Dibujar un corazón en lugar de texto
            center_x, center_y = x + key_size // 2, y + key_size // 2
            cv2.ellipse(frame, (center_x - 10, center_y - 10), (10, 15), 0, 0, 180, (255, 255, 255), -1)
            cv2.ellipse(frame, (center_x + 10, center_y - 10), (10, 15), 0, 0, 180, (255, 255, 255), -1)
            cv2.fillPoly(
                frame,
                [np.array([(int(center_x - 20), int(center_y)), 
                           (int(center_x), int(center_y + 30)), 
                           (int(center_x + 20), int(center_y))], dtype=np.int32)],
                color=(255, 255, 255)
            )
        else:
            cv2.putText(frame, key, (x + 10, y + 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)
        keyboard.append({"key": key, "top_left": top_left, "bottom_right": bottom_right})
    return keyboard

# Capturar video
cap = cv2.VideoCapture(0)
typed_text = ""  # Texto escrito por el usuario
last_key_pressed = None  # Almacenar la última tecla presionada

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    h, w, _ = frame.shape
    original_frame = frame.copy()  # Guardar una copia del frame original

    # Dibujar el teclado
    keyboard = draw_keyboard(frame)

    # Dibujar el texto escrito en la parte superior
    cv2.putText(frame, typed_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)

    # Procesar detección de manos
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks = hand_landmarks.landmark
            index_tip = (int(landmarks[8].x * w), int(landmarks[8].y * h))  # Punta del índice
            thumb_tip = (int(landmarks[4].x * w), int(landmarks[4].y * h))  # Punta del pulgar

            # Dibujar un círculo en la punta del índice
            cv2.circle(frame, index_tip, 10, (255, 255, 255), -1)

            # Calcular distancia entre índice y pulgar
            distance = calculate_distance(index_tip, thumb_tip)

            # Verificar si está presionando una tecla
            if distance < 30:  # Umbral para "presionar" una tecla
                for key in keyboard:
                    top_left = key["top_left"]
                    bottom_right = key["bottom_right"]
                    if top_left[0] <= index_tip[0] <= bottom_right[0] and top_left[1] <= index_tip[1] <= bottom_right[1]:
                        if last_key_pressed == key["key"]:
                            #typed_text += " "  # Agregar un espacio si se repite la tecla
                            continue
                        else:
                            typed_text += key["key"]  # Agregar la tecla al texto
                        last_key_pressed = key["key"]  # Actualizar la última tecla presionada
                        break

    # Mostrar el frame
    cv2.imshow('Virtual Keyboard', frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Salir con ESC
        break

cap.release()
cv2.destroyAllWindows()


## Agregando texto a lo escrito

In [6]:
import cv2
import mediapipe as mp
import math
import numpy as np

# Inicializar MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

# Función para calcular la distancia euclidiana
def calculate_distance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def draw_keyboard(frame, key_size=50):
    h, w, _ = frame.shape
    keys = "ABCDEFGHIJKLMNOPQRSTUVWXYZ<3 "
    keyboard = []
    start_x, start_y = 30, h - 200  # Coordenadas iniciales del teclado
    for i, key in enumerate(keys):
        x = start_x + (i % 10) * (key_size + 5)  # 10 teclas por fila
        y = start_y + (i // 10) * (key_size + 5)
        top_left = (x, y)
        bottom_right = (x + key_size, y + key_size)
        cv2.rectangle(frame, top_left, bottom_right, (255, 255, 255), 2)
        if key == "❤":
            # Dibujar un corazón en lugar de texto
            center_x, center_y = x + key_size // 2, y + key_size // 2
            cv2.ellipse(frame, (center_x - 10, center_y - 10), (10, 15), 0, 0, 180, (255, 255, 255), -1)
            cv2.ellipse(frame, (center_x + 10, center_y - 10), (10, 15), 0, 0, 180, (255, 255, 255), -1)
            cv2.fillPoly(
                frame,
                [np.array([(int(center_x - 20), int(center_y)), 
                           (int(center_x), int(center_y + 30)), 
                           (int(center_x + 20), int(center_y))], dtype=np.int32)],
                color=(255, 255, 255)
            )
        else:
            cv2.putText(frame, key, (x + 10, y + 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)
        keyboard.append({"key": key, "top_left": top_left, "bottom_right": bottom_right})
    return keyboard

# Capturar video
cap = cv2.VideoCapture(0)
typed_text = ""  # Texto escrito por el usuario
last_key_pressed = None  # Almacenar la última tecla presionada
special_text_added = False  # Flag para agregar el texto especial "y victor" una vez

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    h, w, _ = frame.shape
    original_frame = frame.copy()  # Guardar una copia del frame original

    # Dibujar el teclado
    keyboard = draw_keyboard(frame)

    # Dibujar el texto escrito en la parte superior
    cv2.putText(frame, typed_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
    if special_text_added:
        cv2.putText(frame, "y victor x)", (100, 150), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)

    # Dibujar un rectángulo en la esquina superior derecha
    cv2.rectangle(frame, (w - 180, 30), (w - 10, 70), (255, 255, 255), 2)
    cv2.putText(frame, "completo", (w - 160, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Procesar detección de manos
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks = hand_landmarks.landmark
            index_tip = (int(landmarks[8].x * w), int(landmarks[8].y * h))  # Punta del índice

            # Dibujar un círculo en la punta del índice
            cv2.circle(frame, index_tip, 10, (255, 255, 255), -1)

            # Verificar si el índice está dentro del rectángulo
            if w - 150 <= index_tip[0] <= w - 10 and 30 <= index_tip[1] <= 70:
                if not special_text_added:
                    nuevo = " con pollo"  # Agregar el texto especial solo una vez
                    special_text_added = True

            # Calcular distancia entre índice y pulgar
            distance = calculate_distance(index_tip, (int(landmarks[4].x * w), int(landmarks[4].y * h)))

            # Verificar si está presionando una tecla
            if distance < 30:  # Umbral para "presionar" una tecla
                for key in keyboard:
                    top_left = key["top_left"]
                    bottom_right = key["bottom_right"]
                    if top_left[0] <= index_tip[0] <= bottom_right[0] and top_left[1] <= index_tip[1] <= bottom_right[1]:
                        if last_key_pressed == key["key"]:
                            #typed_text += " "  # Agregar un espacio si se repite la tecla
                            continue
                        else:
                            typed_text += key["key"]  # Agregar la tecla al texto
                        last_key_pressed = key["key"]  # Actualizar la última tecla presionada
                        break

    # Mostrar el frame
    cv2.imshow('Virtual Keyboard', frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Salir con ESC
        break

cap.release()
cv2.destroyAllWindows()


## Básico mostrando coordenadas

In [12]:
# Inicialización
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_height, frame_width, _ = frame.shape

    coord_canvas = np.zeros_like(frame)

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)

    # Procesar las manos detectadas
    if results.multi_hand_landmarks:
        for hand_idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            for idx, landmark in enumerate(hand_landmarks.landmark):
                # Obtener las coordenadas relativas al frame
                cx_frame, cy_frame = int(landmark.x * frame_width), int(landmark.y * frame_height)

                # Dibujar los puntos en la imagen principal
                cv2.circle(frame, (cx_frame, cy_frame), 5, (0, 255, 0), -1)
                cv2.putText(frame, str(idx), (cx_frame + 5, cy_frame - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)

                # Dibujar los puntos en el lienzo negro
                cv2.circle(coord_canvas, (cx_frame, cy_frame), 5, (0, 255, 0), -1)
                coord_text = f"{idx}: ({cx_frame}, {cy_frame})"
                cv2.putText(coord_canvas, coord_text, (cx_frame + 10, cy_frame + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

    # Mostrar el video con las manos detectadas y puntos clave
    cv2.imshow('Hand Detection', frame)

    # Mostrar la ventana con puntos clave y coordenadas
    cv2.imshow('Key Points and Coordinates', coord_canvas)

    # Salir con ESC
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Inicialización
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Obtener dimensiones del frame
    frame_height, frame_width, _ = frame.shape

    # Crear un lienzo negro del mismo tamaño que el frame
    coord_canvas = np.zeros_like(frame)

    # Conversión a RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Procesar el frame
    results = hands.process(frame_rgb)

    # Procesar las manos detectadas
    if results.multi_hand_landmarks:
        for hand_idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            for idx, landmark in enumerate(hand_landmarks.landmark):
                # Obtener las coordenadas relativas al frame
                cx_frame, cy_frame = int(landmark.x * frame_width), int(landmark.y * frame_height)

                # Dibujar los puntos en la imagen principal
                cv2.circle(frame, (cx_frame, cy_frame), 5, (0, 255, 0), -1)
                cv2.putText(frame, str(idx), (cx_frame + 5, cy_frame - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)

                # Dibujar los puntos en el lienzo negro
                cv2.circle(coord_canvas, (cx_frame, cy_frame), 5, (0, 255, 0), -1)
                coord_text = f"{idx}: ({cx_frame}, {cy_frame})"
                cv2.putText(coord_canvas, coord_text, (cx_frame + 10, cy_frame + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

    # Combinar las dos imágenes horizontalmente
    combined_frame = np.hstack((frame, coord_canvas))

    # Mostrar la ventana combinada
    cv2.imshow('Hand Detection and Key Points', combined_frame)

    # Salir con ESC
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
